In [30]:
%run web_scrape_func_def.ipynb

imports loaded.

> file management func:

    get_content(directory)
    get_last_update(file_path)

> fighters_stats_scrape func:

    get_xml(url, headers = {'User-Agent':'Moz...}) # Return web page data in .xml format.
    get_links(url)
    get_fighter_data(url) # Retrive data as list
    get_current_date()

> matches_stats_web_scrape func:

    get_event_date(xml)
    get_event_data(event_link)
    get_later_date(date1, date2)


# Main( )

#### Updating variables:

In [41]:
files = get_content(directory='../data/')
last_file_update = None

for file in files:
    if 'ufc_matches_stats' in file:
        last_file_update = get_last_update(file_path = '../data/' + file)

print('Last update: ' + last_file_update)

Last update: 05-04-2023


In [25]:
events_list = 'http://ufcstats.com/statistics/events/completed?page=all'
events = get_links(url=events_list)

#### Extracting data to data frame:

In [35]:
data = []
column_names = ['result','fighter','opponent',
                'f_kd','o_kd','f_str','o_str','f_td','o_td','f_sub','o_sub',
                'weight_class','method','type','round','time','date'
               ]

start_time = time()

for url in tqdm(events):
    data.extend(get_event_data(event_link=url))

end_time = time()        
print('Process time = %f sec' %(end_time - start_time))     
df = pd.DataFrame(data=data, columns=column_names)  
df.tail(3)

100%|████████████████████████████████████████████████████████████████████████████████| 640/640 [06:07<00:00,  1.74it/s]

0 Event added (367.266385 sec)


,result,fighter,opponent,f_kd,o_kd,f_str,o_str,f_td,o_td,f_sub,o_sub,weight_class,method,type,round,time,date
6417,win,Frank Hamaker,Thaddeus Luster,0,0,2,0,1,0,3,0,Open Weight,SUB,Keylock,1,4:52,11-03-1994
6418,win,Johnny Rhodes,David Levicki,0,0,11,4,1,0,0,0,Open Weight,KO/TKO,Punches,1,12:13,11-03-1994
6419,win,Patrick Smith,Ray Wizard,0,0,1,1,0,0,1,0,Open Weight,SUB,Guillotine Choke,1,0:58,11-03-1994


#### Sort by date:

In [40]:
df['date'] = pd.to_datetime(df.date, format='%d-%m-%Y') 
df = df.sort_values(by='date', ascending=False).reset_index(drop=True)
df.tail(3)

,result,fighter,opponent,f_kd,o_kd,f_str,o_str,f_td,o_td,f_sub,o_sub,weight_class,method,type,round,time,date
6417,win,Ken Shamrock,Christophe Leninger,0,0,8,1,1,0,0,0,Open Weight,KO/TKO,Punches,1,4:49,1994-09-09
6418,win,Harold Howard,Roland Payne,1,0,9,3,0,1,0,0,Open Weight,KO/TKO,Punches,1,0:46,1994-09-09
6419,win,Royce Gracie,Kimo Leopoldo,0,0,2,6,0,1,1,0,Open Weight,SUB,Armbar,1,4:40,1994-09-09


#### Saving file:

In [43]:
df.to_csv('../data/ufc_matches_stats.csv', index=0)
print('../data/ufc_matches_stats.csv saved.')

../data/ufc_matches_stats.csv saved.


# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>